In [52]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [53]:
epochs = 100
batch_size = 32
patience = 25
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [3]:
# Transformada de Fourier
def fft(img):
    img = np.fft.fft2(img)
    img = np.fft.fftshift(img)
    return img

# Inversa (retorna para imagem original)
def ifft(fimg):
    fimg = np.fft.ifftshift(fimg)
    fimg = np.fft.ifft2(fimg)
    return fimg

# Obtém a magnitude da imagem
def mag(img):
    absvalue = np.abs(img)
    magnitude = 20 * np.log(absvalue)
    return magnitude

# Normaliza a imagem entre 0 e 255
def norm(img):
    img = cv.normalize(
    img, None, 0, 255,
    cv.NORM_MINMAX
    )

# Melhor para ver imagens da transformada e imagens pequenas em geral.
def show(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    return img


In [54]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Conv2D(62, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(124, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(248,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(124,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 batch_normalization_3 (Bat  (None, 30, 30, 32)        128       
 chNormalization)                                                
                                                      

In [55]:
train = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img", 
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)


Found 6820 files belonging to 62 classes.
Using 5456 files for training.
Found 6820 files belonging to 62 classes.
Using 1364 files for validation.


In [57]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/100


171/171 [==============================] - 13s 78ms/step - loss: 0.3905 - accuracy: 0.8739 - val_loss: 0.7321 - val_accuracy: 0.7999
Epoch 2/100
171/171 [==============================] - 13s 78ms/step - loss: 0.3729 - accuracy: 0.8796 - val_loss: 0.7139 - val_accuracy: 0.8028
Epoch 3/100
171/171 [==============================] - 14s 79ms/step - loss: 0.3748 - accuracy: 0.8748 - val_loss: 0.7301 - val_accuracy: 0.8123
Epoch 4/100
171/171 [==============================] - 13s 78ms/step - loss: 0.3867 - accuracy: 0.8717 - val_loss: 0.7415 - val_accuracy: 0.7999
Epoch 5/100
171/171 [==============================] - 14s 80ms/step - loss: 0.3827 - accuracy: 0.8783 - val_loss: 0.7172 - val_accuracy: 0.8101
Epoch 6/100
171/171 [==============================] - 14s 81ms/step - loss: 0.3827 - accuracy: 0.8785 - val_loss: 0.7795 - val_accuracy: 0.7911
Epoch 7/100
171/171 [==============================] - 14s 79ms/step - loss: 0.3683 - accuracy: 0.8825 - val_loss: 0.7741 - val_accuracy: 0.80